# **INSTALL LIBRARY YANG DIBUTUHKAN**

In [1]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets
!pip install GPUtil
!pip install groq
!pip install rouge_score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 761.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

# **DATASET PREPARATION**

In [ ]:
import os
import re
import json
from groq import Groq

# 1. Load API key from environment (or use default for development)
API_KEY = os.getenv("GROQ_API_KEY", "YOUR_API_KEY_HERE")
client = Groq(api_key=API_KEY)
MODEL = "llama-3.3-70b-versatile"

# 2. Seed examples (few-shot)
seed_examples = [
    {
        "input": "How do I reset my password in the Bajau E-Commerce mobile app?",
        "output": (
            "1. Open the Bajau app\n"
            "2. Tap “Forgot Password” on the login screen\n"
            "3. Enter your email\n"
            "4. Tap “Submit” to get reset link\n"
            "5. Check your inbox and click the link\n"
            "6. Enter & confirm new password\n"
            "7. Tap “Save”"
        )
    },
    {
        "input": "How do I create a new account in the Bajau E-Commerce mobile app?",
        "output": (
            "1. Open the Bajau app\n"
            "2. Tap “Sign Up”\n"
            "3. Enter name, email, password\n"
            "4. Agree Terms of Service\n"
            "5. Tap “Create Account”\n"
            "6. Verify email via link\n"
            "7. Log in with new credentials"
        )
    },
    {
        "input": "How do I use the chatbot in the Bajau E-Commerce mobile app?",
        "output": (
            "1. Open the Bajau app\n"
            "2. Tap chat icon\n"
            "3. Type your question\n"
            "4. Press “Send”\n"
            "5. Read bot’s reply\n"
            "6. Provide any extra details\n"
            "7. Tap “Escalate” to talk with agent"
        )
    },
    # Tambahkan contoh tambahan jika diinginkan
]

# 3. Build the conversation messages for few-shot
messages = [
    {
        "role": "system",
        "content": (
            "You are an assistant that ONLY replies with a JSON array of objects, "
            "each having `input` (a user question) and `output` (numbered steps). "
            "No extra text, no markdown fences."
        )
    }
]

for ex in seed_examples:
    messages.append({
        "role": "user",
        "content": json.dumps([ex], ensure_ascii=False)
    })
    messages.append({
        "role": "assistant",
        "content": json.dumps([ex], ensure_ascii=False)
    })

# 4. Final user prompt to generate 100 new examples
messages.append({
    "role": "user",
    "content": (
        "PT Bajau Escorindo is one of the forefront of IT innovation and Solutions company in Indonesia."
        "Bajau just release it's own E-Commerce application named Bajau E-Commerce."
        "Now generate 150 new examples following exactly that JSON-array format. "
        "OUTPUT MUST BE A PURE JSON ARRAY—no extra text or fences."
    )
})

# 5. Call the API
resp = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    max_tokens=15000,
    temperature=0.7,
    n=1,
    stop=["]"],  # stop at end of JSON array
)

raw = resp.choices[0].message.content

In [3]:
# 6. Clean up the raw text to extract the JSON array
clean = raw.strip()
if clean.startswith("```"):
    clean = clean.split("```", 2)[1].rsplit("```", 1)[0]
match = re.search(r'(\[.*)$', clean, flags=re.DOTALL)
if not match:
    raise ValueError("Cannot find JSON array in model output.")
clean = match.group(1)
if not clean.rstrip().endswith("]"):
    clean = clean.rstrip() + "]"

# 7. Repair by extracting well-formed JSON objects
object_pattern = r'\{(?:[^{}]|"(?:\\.|[^"\\])*")*\}'
objects = re.findall(object_pattern, clean, flags=re.DOTALL)
repaired_json = "[" + ",".join(objects) + "]"

# 8. Parse the repaired JSON
try:
    data = json.loads(repaired_json)
except json.JSONDecodeError as e:
    print("Failed to parse JSON after repair:", e)
    print(repaired_json[:500])
    raise

# 9. Save to file
output_file = "dataset.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Success: {len(data)} data written to '{output_file}'.")

Success: 186 data written to 'dataset.json'.


In [4]:
print(data[0])

{'input': 'How do I track my order in the Bajau E-Commerce mobile app?', 'output': '1. Open the Bajau app\n2. Tap “My Account”\n3. Select “My Orders”\n4. Choose the order to track\n5. Tap “Track Order”\n6. View order status\n7. Contact support if needed'}


In [5]:
import json

# Load the data from dataset.json
try:
    with open("dataset.json", "r", encoding="utf-8") as f:
        dataset = json.load(f)

    # Print the first 5 examples
    print("--- First 5 Examples from dataset.json ---")
    for i, example in enumerate(dataset[:5]):
        print(f"Example {i+1}:")
        print(json.dumps(example, indent=2, ensure_ascii=False))
        print("-" * 20)

except FileNotFoundError:
    print("Error: dataset.json not found. Please ensure the previous cell ran successfully to create the file.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON from dataset.json: {e}")

--- First 5 Examples from dataset.json ---
Example 1:
{
  "input": "How do I track my order in the Bajau E-Commerce mobile app?",
  "output": "1. Open the Bajau app\n2. Tap “My Account”\n3. Select “My Orders”\n4. Choose the order to track\n5. Tap “Track Order”\n6. View order status\n7. Contact support if needed"
}
--------------------
Example 2:
{
  "input": "What payment methods are available in the Bajau E-Commerce mobile app?",
  "output": "1. Open the Bajau app\n2. Tap “My Account”\n3. Select “Payment Methods”\n4. View available payment options\n5. Add new payment method\n6. Save changes\n7. Proceed to checkout"
}
--------------------
Example 3:
{
  "input": "How do I return a product in the Bajau E-Commerce mobile app?",
  "output": "1. Open the Bajau app\n2. Tap “My Account”\n3. Select “My Orders”\n4. Choose the order to return\n5. Tap “Return Item”\n6. Follow return instructions\n7. Wait for refund or replacement"
}
--------------------
Example 4:
{
  "input": "Can I change my o

# **DATASET PREPROCESSING**

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import gc
model_ref = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Define the model name
model_name = model_ref

# Load pre-trained model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training data size: {len(train_data)}")
print(f"Testing data size: {len(test_data)}")

Training data size: 148
Testing data size: 38


In [8]:
# Data preprocessing, Tokenization, LoRA configuration and training arguments
def format_qa(example):
    return {
        "text": f"Question: {example['input']} Answer: {example['output']}"
    }

qa_train_dataset = Dataset.from_list(train_data)
formatted_train_dataset = qa_train_dataset.map(format_qa)

qa_test_dataset = Dataset.from_list(test_data)
formatted_test_dataset = qa_test_dataset.map(format_qa)

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [9]:
# Tokenization
def preprocess_function(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # Labels should be a shifted version of input_ids for causal LM training
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply tokenization
tokenized_train_dataset = formatted_train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = formatted_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

# **FINE-TUNE MODEL DENGAN METODE LORA PEFT**

Base Model: TinyLlama-1.1B

In [10]:
# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

In [11]:
# Set Training Arguments

training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Adjusted for GPU memory limitations
    gradient_accumulation_steps=8,  # To simulate a larger batch size
    warmup_steps=100,
    max_steps=250,
    learning_rate=2e-4,
    fp16=True,  # Enable mixed precision training
    logging_steps=10,
    output_dir="outputs",
    report_to="none",
    remove_unused_columns=False,
)

In [12]:
# Model fine-tune and save

# Move model to CPU to free memory before training
model = model.to("cpu")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
# Free up memory before training
gc.collect()  # Garbage collection
torch.cuda.empty_cache()  # Clears CUDA cache
print("GPU clache cleared")

GPU clache cleared


In [14]:
# Optimize model with torch.compile (improves execution speed)
model = torch.compile(model)

# Move model back to GPU for training
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [15]:
# Start training
trainer.train()

# Save the fine tuned model
model.save_pretrained("fine-tuned-QA-tinyllama-1.1B")
tokenizer.save_pretrained("fine-tuned-QA-tinyllama-1.1B")

Step,Training Loss
10,14.781300
20,9.759900
30,1.600000
40,0.276300
50,0.154800
60,0.069200
70,0.035700
80,0.020800
90,0.018200
100,0.016000


('fine-tuned-QA-tinyllama-1.1B/tokenizer_config.json',
 'fine-tuned-QA-tinyllama-1.1B/special_tokens_map.json',
 'fine-tuned-QA-tinyllama-1.1B/chat_template.jinja',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.model',
 'fine-tuned-QA-tinyllama-1.1B/added_tokens.json',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.json')

In [16]:
# Optional: Merge adapter with base model
merged_model = model.merge_and_unload()  # This creates a full model: base + adapter

# Save the full model
merged_model.save_pretrained("final-model-merged")
tokenizer.save_pretrained("final-model-merged")

('final-model-merged/tokenizer_config.json',
 'final-model-merged/special_tokens_map.json',
 'final-model-merged/chat_template.jinja',
 'final-model-merged/tokenizer.model',
 'final-model-merged/added_tokens.json',
 'final-model-merged/tokenizer.json')

In [16]:
# Load the fine-tuned model
model_path = "fine-tuned-QA-tinyllama-1.1B"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_answer(question, max_length=256):
    prompt = f"Question: {question}\nAnswer:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_k=50, top_p=0.9)

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [17]:
# Example Test Cases - Known Questions
test_questions = [
    "How do I exchange a bundle order for a different bundle in the Bajau E-Commerce mobile app?",
    "What is the process for using a promo code to purchase a subscription in the Bajau E-Commerce mobile app?",
    "How do I access my subscription order history in the Bajau E-Commerce mobile app?",
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {generate_answer(q)}\n")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: How do I exchange a bundle order for a different bundle in the Bajau E-Commerce mobile app?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: Question: How do I exchange a bundle order for a different bundle in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “My Account”
3. Tap “Order History”
4. Select the bundle order
5. Tap “Exchange”
6. Select new bundle
7. Contact support with questions

Q: What is the process for using a promo code to purchase a subscription in the Bajau E-Commerce mobile app?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: Question: What is the process for using a promo code to purchase a subscription in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “Subscription”
3. Tap “Promo Code”
4. Enter promo code
5. Apply promo code
6. Purchase subscription
7. View subscription details

Q: How do I access my subscription order history in the Bajau E-Commerce mobile app?
A: Question: How do I access my subscription order history in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “My Account”
3. Tap “Subscription Order History”
4. View subscription order history
5. Select order to view details
6. View order status
7. Contact support with questions



In [18]:
# Example Test Cases - Known Questions (Paraphrased)
test_questions = [
    "How can I swap a bundle order for a different bundle in the Bajau E-Commerce mobile app?",
    "What steps do I need to follow to apply a promo code when purchasing a subscription in the Bajau E-Commerce mobile app?",
    "How can I view my subscription order history in the Bajau E-Commerce mobile app?",
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {generate_answer(q)}\n")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: How can I swap a bundle order for a different bundle in the Bajau E-Commerce mobile app?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: Question: How can I swap a bundle order for a different bundle in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “My Account”
3. Tap “Order History”
4. Select the bundle order
5. Tap “Swap for Another Bundle”
6. Select the new bundle order
7. Contact support with questions

Q: What steps do I need to follow to apply a promo code when purchasing a subscription in the Bajau E-Commerce mobile app?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: Question: What steps do I need to follow to apply a promo code when purchasing a subscription in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “Subscription”
3. Tap “Promo Code”
4. View promo code details
5. Select subscription to apply promo code
6. Apply promo code
7. View subscription details

Q: How can I view my subscription order history in the Bajau E-Commerce mobile app?
A: Question: How can I view my subscription order history in the Bajau E-Commerce mobile app?
Answer:
1. Open the Bajau app
2. Tap “Subscription Order History”
3. View order history
4. Select subscription
5. View order details
6. View order history
7. Contact support with questions



In [19]:
# Example Test Cases - Unknown Questions (Tidak ada di dataset)
test_questions = [
    "How to Log Out from Bajau App?",
    "How to Register to Bajau App?",
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {generate_answer(q)}\n")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: How to Log Out from Bajau App?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A: Question: How to Log Out from Bajau App?
Answer:
1. Open the Bajau app
2. Tap “My Account”
3. Tap “Log Out”
4. Confirm log out
5. Exit app
6. Add to home screen
7. Contact support with questions

Q: How to Register to Bajau App?
A: Question: How to Register to Bajau App?
Answer:
1. Open the app
2. Tap “My Account”
3. Tap “Register”
4. Enter name
5. Enter email address
6. Enter mobile number
7. Tap “Create Account”
8. Verify phone number
9. Set up payment method
10. Start using Bajau app



# **LOAD DAN EVALUASI METRIK MODEL**


In [20]:
import torch
from tqdm.auto import tqdm
import evaluate

# Load evaluation metrics with the new API
rouge_metric = evaluate.load("rouge")
bleu_metric  = evaluate.load("bleu")

# Function to generate predictions
def generate_prediction(question, model, tokenizer, device, max_length=256):
    prompt = f"Question: {question}\nAnswer:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            top_k=50,
            top_p=0.9
        )
    # Decode only the generated part
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Strip off the prompt
    return generated_text.replace(prompt, "").strip()

# Generate predictions
predictions = []
references  = []

for example in tqdm(test_data):
    predictions.append(generate_prediction(
        example['input'], model, tokenizer, device
    ))
    references.append(example['output'])

# Compute ROUGE
rouge_results = rouge_metric.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)
print("ROUGE Scores:", rouge_results)

# Compute BLEU (expects list of reference lists)
bleu_references = [[ref] for ref in references]
bleu_results    = bleu_metric.compute(
    predictions=predictions,
    references=bleu_references
)
print("BLEU Score:", bleu_results)


  0%|          | 0/38 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

ROUGE Scores: {'rouge1': np.float64(0.936710671474426), 'rouge2': np.float64(0.8702513910718245), 'rougeL': np.float64(0.9089164219539468), 'rougeLsum': np.float64(0.9342646553889045)}
BLEU Score: {'bleu': 0.8782352519050307, 'precisions': [0.942381562099872, 0.9015748031496063, 0.851278600269179, 0.8225138121546961], 'brevity_penalty': 1.0, 'length_ratio': 1.0006406149903908, 'translation_length': 1562, 'reference_length': 1561}
